In [1]:
using JuMP
using Clp

In [7]:
m = Model(Clp.Optimizer)

@variables(m,begin
    PA_MAR>=0 #Power production -- plant A in March
    PA_APR>=0 #Power production -- plant A in April
    PB_MAR>=0 #Power production -- plant B in March
    PB_APR>=0 #Power production -- plant B in April
    SA_MAR>=0 #Water spilled -- reservior A in March
    SA_APR>=0 #Water spilled -- reservior A in April
    SB_MAR>=0 #Water spilled -- reservior B in March
    SB_APR>=0 #Water spilled -- reservior B in April
    LA_MAR>=0 #Level -- reservior A start of March (exogenous)
    LA_APR>=0 #Level -- reservior A start of April
    LA_MAY>=0 #Level -- reservior A start of May
    LB_MAR>=0 #Level -- reservior A start of March (exogenous)
    LB_APR>=0 #Level -- reservior A start of April
    LB_MAY>=0 #Level -- reservior A start of May
    EH_MAR>=0 #High value electricity generation -- March
    EL_MAR>=0 #Low value electricity generation -- March
    EH_APR>=0 #High value electricity generation -- April
    EL_APR>=0 #Low value electricity generation -- April
end)


# Levels at the beginning of March are specified exogenously:
fix(LA_MAR,1900,force=true)
fix(LB_MAR, 850,force=true)
# Power can be sold at CHF 5 per MWH up to 50,000 MWH per month:
set_upper_bound(EH_MAR, 50000)
set_upper_bound(EH_APR, 50000)
# Storage capacity:
set_upper_bound(LA_APR, 2000)
set_upper_bound(LA_MAY, 2000)
set_upper_bound(LB_APR, 1500)
set_upper_bound(LB_MAY, 1500)
# Minimum reservior levels:
set_lower_bound(LA_APR, 1200)
set_lower_bound(LA_MAY, 1200)
set_lower_bound(LB_APR, 800)
set_lower_bound(LB_MAY, 800)
# Power plant capacities:
set_upper_bound(PA_MAR, 60000)
set_upper_bound(PA_APR, 60000)
set_upper_bound(PB_MAR, 35000)
set_upper_bound(PB_APR, 35000)


@objective(m,Max, 5*(EH_MAR+EH_APR) + 3.5*(EL_MAR+EL_APR))

@constraints(m,begin
    sales_mar, EH_MAR + EL_MAR == PA_MAR + PB_MAR
    sales_apr, EH_APR + EL_APR == PA_APR + PB_APR
    
    alevel_apr, LA_APR == LA_MAR + 200 - SA_MAR - PA_MAR / 400
    alevel_may, LA_MAY == LA_APR + 130 - SA_APR - PA_APR / 400
    blevel_apr, LB_APR == LB_MAR + 40 + SA_MAR - SB_MAR - PB_MAR / 200
    blevel_may, LB_MAY == LB_APR + 15 + SA_APR - SB_APR - PB_APR / 200
end)

optimize!(m)

solution_summary(m)

Coin0506I Presolve 6 (0) rows, 12 (-6) columns and 20 (-6) elements
Clp0006I 0  Obj -0 Primal inf 34906.094 (1) Dual inf 35.950619 (4)
Clp0006I 9  Obj 815000
Clp0000I Optimal - objective value 815000
Coin0511I After Postsolve, objective 815000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 815000 - 9 iterations time 0.002, Presolve 0.00


* Solver : Clp

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "0 - optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.15000e+05

* Work counters
  Solve time (sec)   : 0.00000e+00


In [9]:
for var in all_variables(m)
    println("$var = $(value(var))")
end

PA_MAR = 60000.0
PA_APR = 60000.0
PB_MAR = 34999.99999999999
PB_APR = 34999.99999999999
SA_MAR = 730.0
SA_APR = 0.0
SB_MAR = 485.0
SB_APR = 0.0
LA_MAR = 1900.0
LA_APR = 1220.0
LA_MAY = 1200.0
LB_MAR = 850.0
LB_APR = 960.0
LB_MAY = 800.0
EH_MAR = 50000.0
EL_MAR = 44999.99999999999
EH_APR = 50000.0
EL_APR = 44999.99999999999
